# Spark Context

In [3]:
!pip install elasticsearch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.2/480.2 kB 4.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.5/64.5 kB 3.2 MB/s eta 0:00:00


In [1]:
from pyspark.sql import SparkSession

# Spark session & context
spark = (SparkSession
         .builder
         .master("local")
         .appName("es-test")
         .config("spark.driver.extraClassPath", "/root/project/de-2024/resources/elasticsearch-spark-30_2.12-8.4.3.jar")
         .config("spark.jars", "/root/project/de-2024/resources/elasticsearch-spark-30_2.12-8.4.3.jar")
         .getOrCreate())
sc = spark.sparkContext

your 131072x1 screen size is bogus. expect trouble
24/07/03 20:20:08 WARN Utils: Your hostname, DESKTOP-RMO2CFC resolves to a loopback address: 127.0.1.1; using 172.18.9.25 instead (on interface eth0)
24/07/03 20:20:08 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
24/07/03 20:20:11 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [2]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = \
  '--jars /root/project/de-2024/resources/elasticsearch-spark-30_2.12-8.4.3.jar pyspark-shell'

In [5]:
df = spark.createDataFrame([(1, "Hello"), (2, "World")], ["id", "message"])
df.show()

In [ ]:
df.write.format("org.elasticsearch.spark.sql") \
    .mode("append") \
    .option("es.nodes", "http://localhost:9200") \
    .option("es.nodes.discovery", "true") \
    .option("es.resource", "daily-stats-2024") \
    .save("test")
spark.stop()

In [4]:
from elasticsearch import Elasticsearch

# ElasticSearch 클라이언트 생성
es = Elasticsearch("http://localhost:9200")


In [7]:
# 모든 인덱스 조회
indices = es.indices.get_alias("*")
print("Available indices:", list(indices.keys()))


Available indices: ['daily-stats-2024']


In [ ]:
  query = {
            "size" : 1, 
            "sort" : [
                {
                    "stockdate" : {"order" : 'desc'} 
                }  
            ],
            "query" : {
                "term" : {"code":code}
                       },
            "_Source" : ["crawl_tbl"]
        }
        response = self.es.search(index='crawl_tbl', body = query)
        if response['hits']['hits']: 
            last_date = response['hits']['hits'][0]['_source']['stockdate'] 
            return last_date 
        else : 
            return None 
        

In [14]:


query = {
            "size" : 1, 
            "sort" : [
                {
                    "stockdate" : {"order" : 'desc'} 
                }  
            ],
            "query" : {
                "term" : {"code":'00020'}
                       },
            "_source" : ["crawl_tbl"]
        }
response = es.search(index='crawl_tbl', body = query)

BadRequestError: BadRequestError(400, 'search_phase_execution_exception', 'No mapping found for [stockdate] in order to sort on')

In [15]:
es.indices.delete(index='crawl_tbl')

ObjectApiResponse({'acknowledged': True})

In [9]:
if not es.indices.exists(index='crawl_tbl') : 
    es.indices.create(index='crawl_tbl', body = {
        "settings" : { 
            'number_of_shards' : 1 , 
            'number_of_replicas' : 1
            }
    })

In [11]:
es.indices.exists(index='crawl_tbl')

HeadApiResponse(True)

In [10]:
# 특정 인덱스의 모든 데이터 조회
response = es.search(index="crawl_tbl", body={"query": {"match_all": {}}})

# 검색 결과 출력
for hit in response['hits']['hits']:
    print(hit['_source'])
